# Starter API Access Pipeline
---

## Imports

In [472]:
# Dependencies
# import datetime as dt
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

import havoc_helper as havoc
import api_keys

In [473]:
fema_decl = pd.read_csv('Assets/CSVs/DisasterDeclarationsSummaries.csv', low_memory = False)
fema_decl.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5530-NV,5530,NV,FM,2024-08-12T00:00:00.000Z,2024,Fire,GOLD RANCH FIRE,0,0,...,99031,Washoe (County),24123,NaN,2024081201,9,R,2024-08-27T18:22:14.800Z,5d07e7c51bb300bfbec94a699a1e1ab1d61a97cd,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,FM-5529-OR,5529,OR,FM,2024-08-09T00:00:00.000Z,2024,Fire,LEE FALLS FIRE,0,0,...,99067,Washington (County),24122,NaN,2024081001,10,R,2024-08-27T18:22:14.800Z,ae87cf3c6ed795015b714af7166c7c295b2b67c7,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,FM-5528-OR,5528,OR,FM,2024-08-06T00:00:00.000Z,2024,Fire,ELK LANE FIRE,0,0,...,99031,Jefferson (County),24116,NaN,2024080701,10,R,2024-08-27T18:22:14.800Z,432cf0995c47e3895cea696ede5621b810460501,59983f89-30bf-4888-b21b-62e8d57d9aac
3,FM-5527-OR,5527,OR,FM,2024-08-02T00:00:00.000Z,2024,Fire,MILE MARKER 132 FIRE,0,0,...,99017,Deschutes (County),24111,NaN,2024080301,10,R,2024-08-27T18:22:14.800Z,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,FM-5522-CA,5522,CA,FM,2024-07-27T00:00:00.000Z,2024,Fire,BOREL FIRE,0,0,...,99029,Kern (County),24102,NaN,2024072701,9,R,2024-08-27T18:22:14.800Z,51ec819011c936f387edf10b9d5839b35419ca95,9b73e19b-d326-4992-8da1-7e658d97607c


In [474]:
# Removing unwanted/unusable columns
fema_decl_clean = fema_decl[
    [
        'disasterNumber', 'declarationRequestNumber', 'declarationTitle', 'incidentType', 
        'declarationType', 'designatedArea', 'state', 'region', 'declarationDate', 
        'incidentBeginDate', 'tribalRequest', 'ihProgramDeclared', 'iaProgramDeclared', 
        'paProgramDeclared', 'hmProgramDeclared', 'id'
    ]
].copy()
fema_decl_clean['declarationDate'] = fema_decl_clean[
    'declarationDate'
    ].str.slice_replace(10)
fema_decl_clean['incidentBeginDate'] = fema_decl_clean[
    'incidentBeginDate'
    ].str.slice_replace(10)
display(fema_decl_clean)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,id
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1,59983f89-30bf-4888-b21b-62e8d57d9aac
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1,9b73e19b-d326-4992-8da1-7e658d97607c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67351,9,53010,FLOOD,Flood,DR,Statewide,TX,6,1953-06-19,1953-06-19,0,0,1,1,1,6ef68fa6-5889-466b-9e3b-ee4c06da0876
67352,8,53008,FLOOD,Flood,DR,Statewide,IA,7,1953-06-11,1953-06-11,0,0,1,1,1,f100b5c1-0f3f-4c3d-ad3c-50040e98ed8d
67353,7,53009,TORNADO,Tornado,DR,Statewide,MA,1,1953-06-11,1953-06-11,0,0,1,1,1,fca83efc-524a-45a0-ac98-2d43b954a5ea
67354,2,53003,TORNADO & HEAVY RAINFALL,Tornado,DR,Statewide,TX,6,1953-05-15,1953-05-15,0,0,1,1,1,ff821327-6b90-4246-b19f-fff8c4b288a8


In [475]:
# Unique FEMA Disaster Types
disaster_types = fema_decl_clean['incidentType'].unique()
pprint(disaster_types)

array(['Fire', 'Severe Storm', 'Straight-Line Winds', 'Flood',
       'Winter Storm', 'Hurricane', 'Tornado', 'Mud/Landslide',
       'Tropical Storm', 'Snowstorm', 'Earthquake', 'Coastal Storm',
       'Other', 'Severe Ice Storm', 'Biological', 'Dam/Levee Break',
       'Typhoon', 'Volcanic Eruption', 'Freezing', 'Toxic Substances',
       'Chemical', 'Terrorist', 'Drought', 'Human Cause',
       'Fishing Losses', 'Tsunami'], dtype=object)


In [476]:
# TODO -> Remove volcanoes, earthquake, mud/landslide
disaster_natural = pd.DataFrame(
    {
        'incidentType': [
            'Fire', 'Severe Storm', 'Straight-Line Winds', 
            'Flood', 'Winter Storm', 'Hurricane', 'Tornado',
            'Mud/Landslide', 'Tropical Storm', 'Snowstorm', 
            'Earthquake', 'Coastal Storm', 'Severe Ice Storm', 
            'Typhoon', 'Volcanic Eruption', 'Freezing', 
            'Drought', 'Tsunami'
        ]
    }
)

In [477]:
# Filter for just natural disaster types
fema_disasters = pd.merge(fema_decl_clean, disaster_natural, how = 'right', on = 'incidentType')

# TODO: Do we want renamed columns? Or do we like the current columns?
# fema_disasters = fema_disasters.rename(
#     columns = {
#         'disasterNumber': 'disaster_number', 'designatedArea': 'area',
#         'declarationRequestNumber': 'request_number', 'declarationTitle': 'title',
#         'incidentType': 'disaster_type'

#     }
# )
display(fema_disasters)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,id
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1,59983f89-30bf-4888-b21b-62e8d57d9aac
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1,9b73e19b-d326-4992-8da1-7e658d97607c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59096,1967,11042,TSUNAMI WAVES,Tsunami,DR,Honolulu (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1,9ce5ad8d-f0e1-4d12-9907-a704fe2f4511
59097,1967,11042,TSUNAMI WAVES,Tsunami,DR,Maui (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1,d64d8673-10bc-4799-809b-70317d714d68
59098,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Coos (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1,35f729e0-73ff-42aa-9b80-8bcc901be612
59099,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Curry (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1,89c45999-d563-423f-a2dd-e69b96103b55


In [478]:
# Counts of declaration types 
# DR = Major Disasters, EM = Emergency Declaratation, FM = Fire Management)
# For more info see README (in progress)
fema_disasters['declarationType'].value_counts()

declarationType
DR    41116
EM    15991
FM     1994
Name: count, dtype: int64

In [479]:
# Counts of natural disaster types
fema_disasters['incidentType'].value_counts()

incidentType
Severe Storm           18399
Hurricane              13721
Flood                  11093
Fire                    3759
Snowstorm               3707
Severe Ice Storm        2942
Tornado                 1623
Drought                 1292
Tropical Storm          1047
Coastal Storm            637
Freezing                 301
Earthquake               228
Typhoon                  130
Winter Storm             117
Volcanic Eruption         51
Mud/Landslide             43
Tsunami                    9
Straight-Line Winds        2
Name: count, dtype: int64

In [480]:
# Validating no NaNs
fema_disasters.count()

disasterNumber              59101
declarationRequestNumber    59101
declarationTitle            59101
incidentType                59101
declarationType             59101
designatedArea              59101
state                       59101
region                      59101
declarationDate             59101
incidentBeginDate           59101
tribalRequest               59101
ihProgramDeclared           59101
iaProgramDeclared           59101
paProgramDeclared           59101
hmProgramDeclared           59101
id                          59101
dtype: int64

In [481]:
# Filter: Natural disasters per State
fema_disasters['state'].value_counts()

state
TX    4779
FL    2635
KY    2554
MO    2534
LA    2398
GA    2333
VA    2308
OK    2305
NC    2110
PR    1947
MS    1777
IA    1719
KS    1607
AL    1603
CA    1557
TN    1532
AR    1472
MN    1406
NE    1336
ND    1280
IN    1279
NY    1278
SD    1181
WV    1148
PA    1118
IL    1109
OH    1102
SC    1039
WA     902
WI     725
MI     595
NJ     556
OR     536
CO     531
MT     507
ME     426
MD     399
NM     380
VT     377
MA     365
NH     300
ID     276
AZ     271
CT     245
AK     211
NV     198
UT     183
RI     106
HI      97
WY      86
VI      79
MP      67
AS      66
MH      53
DE      47
FM      31
GU      20
DC      19
PW       1
Name: count, dtype: int64

In [482]:
# Filter: Natural disasters per hit area
fema_disasters['designatedArea'].value_counts()

designatedArea
Washington (County)                          510
Jefferson (County)                           462
Statewide                                    450
Jackson (County)                             450
Franklin (County)                            441
                                            ... 
Southern Ute Indian Reservation                1
Ute Mountain Indian Reservation (Also NM)      1
Nondalton (ANV/ANVSA)                          1
Salt River Pima-Maricopa Indian Community      1
Namorik (County-equivalent)                    1
Name: count, Length: 2201, dtype: int64

## Cleaning Area Data for Geocoding

In [483]:
display(fema_disasters)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,id
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1,59983f89-30bf-4888-b21b-62e8d57d9aac
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1,9b73e19b-d326-4992-8da1-7e658d97607c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59096,1967,11042,TSUNAMI WAVES,Tsunami,DR,Honolulu (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1,9ce5ad8d-f0e1-4d12-9907-a704fe2f4511
59097,1967,11042,TSUNAMI WAVES,Tsunami,DR,Maui (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1,d64d8673-10bc-4799-809b-70317d714d68
59098,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Coos (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1,35f729e0-73ff-42aa-9b80-8bcc901be612
59099,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Curry (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1,89c45999-d563-423f-a2dd-e69b96103b55


In [484]:
# Splitting up area from the areaType in designatedArea
geocode_df = fema_disasters[['designatedArea', 'state', 'id']].copy()
geocode_df[['area', 'areaType']] = geocode_df['designatedArea'].str.split('(', n = 1, expand = True)
geocode_df = geocode_df.drop(axis = 0, columns = 'designatedArea')
areas = geocode_df.pop('area')
areaTypes = geocode_df.pop('areaType')
areaTypes = '(' + areaTypes
geocode_df.insert(0, 'area', areas)
geocode_df.insert(1, 'areaType', areaTypes)

In [485]:
# Regex for cleaning up text in areaType
geocode_df['areaType'] = geocode_df['areaType'].str.extract(r'\(([^)]+)\)')
geocode_df['areaType'] = geocode_df['areaType'].str.replace(r'\b(of|Also|and)\b', '', regex = True).str.strip()
display(geocode_df)

,area,areaType,state,id
0,Washoe,County,NV,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,Washington,County,OR,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,Jefferson,County,OR,59983f89-30bf-4888-b21b-62e8d57d9aac
3,Deschutes,County,OR,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,Kern,County,CA,9b73e19b-d326-4992-8da1-7e658d97607c
...,...,...,...,...
59096,Honolulu,County,HI,9ce5ad8d-f0e1-4d12-9907-a704fe2f4511
59097,Maui,County,HI,d64d8673-10bc-4799-809b-70317d714d68
59098,Coos,County,OR,35f729e0-73ff-42aa-9b80-8bcc901be612
59099,Curry,County,OR,89c45999-d563-423f-a2dd-e69b96103b55


In [486]:
geocode_df['areaType'].unique()

array(['County', nan, 'Parish', 'Borough', 'Census Area',
       'Indian Reservation', 'Municipio', 'ANV/ANVSA', 'NM',
       'County-equivalent', 'OTSA', 'NM  UT', 'Sisseton', 'Reservation',
       'District', 'Island', 'AZ  UT', 'Township',
       'Unorganized Territory', 'Police Jury Ward', 'Nett Lake', 'TDSA',
       'KS', 'Municipality', 'CCD', 'City  Borough',
       'Native Regional Corporation', 'State', 'Census Subarea'],
      dtype=object)

In [487]:
# Setting up the Data Frame containing Area Types with 1
# or more states instead of classic 'County' or 'Reservation'
# --> Result is duplicated incidents but for the additional states

# Multi-state regex location in 'areaType' column
multiState_regex = r'\b[A-Z]{2}(?:\s[A-Z]{2})*\b'
states_to_split = geocode_df['areaType'].str.contains(multiState_regex, na = False)
new_rows = []

# Iterate through just the Area Types that need to be split and duplicated
for index, row in geocode_df[states_to_split].iterrows():
    states = row['areaType'].split()
    for state in states:
        new_row = row.copy()
        new_row['state'] = state
        new_rows.append(new_row)

geocode_df = pd.concat(     # Concatenate duplicated rows for multiple states to the main dataframe
    [geocode_df, pd.DataFrame(new_rows)], ignore_index = True
    ).reset_index(drop = True).copy()

In [488]:
# Set Non-Capture Regex and location rows that can have areaType set to 'Reservation'
reservation_regex = r'(?:Indian)|(?:Reservation)|(?:Tribe)|(?:Trust Lands)|(?:TDSA)'
geocode_df.loc[
    geocode_df['area'].str.contains(
        reservation_regex, case = False, regex = True), 'areaType'
        ] = 'Native Land'

In [489]:
# Nitty-Gritty edits (less than 1000 rows each)
# Null `areaTypes`: Regional Educational Attendence Area, Crow/Northern Cheyenne Area
null_areaType = geocode_df['areaType'].isnull() # Set null `areaType` variable for easy-access

# # Change Puerto Rican spelling of Municipality
# geocode_df.loc[geocode_df['areaType'] == 'Municipio', 'areaType'] = 'Municipality'

# Found under Null types (NaN)
# Location Statewide listed areas, copying abbreviations from `state` column
# to the `area` column, extending them to full length names and then setting 
# the `areaType` to State
statewide_locs = (geocode_df['area'] == 'Statewide')    # Bool location
geocode_df.loc[statewide_locs, 'areaType'] = 'State'    # Set type to State
geocode_df.loc[statewide_locs, 'area'] = geocode_df.loc[statewide_locs, 'state']    # Copy State value to area
geocode_df['area'] = geocode_df['area'].replace(havoc.state_dict)   # Transform abbr states to Full name

# # Found under Null types (NaN) 
# # Less than 20 entries of 'Carson City' -> Correct `areaType` to `City`
# geocode_df.loc[geocode_df['area'] == 'Carson City', 'areaType'] = 'City'

# Using geocode_df['areaType'].unique() location type 'City  Borough'
# Only 1 entry of Wranell, AK (Alaska) -> Correct to 'Borough'
geocode_df.loc[geocode_df['areaType'] == 'City  Borough', 'areaType'] = 'Borough'

# Regional Educational Attendance Area `area` with Null `areaType` (it's the same with or without null spec)
# TODO: Correct to `areaType` -> 'Regional Educational Attendance Area' or 'School District'
# geocode_df.loc[reaa, 'area']
reaa_regex = r'Regional Educational.*$'
reaa_loc = geocode_df['area'].str.contains(reaa_regex, case = False, regex = True)
geocode_df.loc[reaa_loc, 'areaType'] = 'County'
geocode_df.loc[reaa_loc, 'area'] = geocode_df.loc[reaa_loc, 'area'].str.replace(reaa_regex, '', regex = True)

In [490]:
# # Set `areaType` = 'Village' when `area` contains the word 'Pueblo', but not when it's the only word
# pueblo_regex = r'\bPueblo\b'
# pueblo_loc = geocode_df['area'].str.contains(pueblo_regex, case = False, regex = True)
# # geocode_df.loc[pueblo_loc, 'areaType'] = 'Village'

# LAST STEP:
# Null areaType with States in list to compare by
# LARGE NULL GROUP: `areaType` is NaN and `state` is 'VA' 537 Rows
# Correct `areaType` -> 'City'
# states_toCheck = ['IA', 'MD', 'MN', 'MO', 'VA']
# geocode_df.loc[((null_areaType) & (
#     (geocode_df['state'].isin(states_toCheck)))),
#     'areaType'] = 'City'

# country_toCheck = ['FM', 'MH']
# geocode_df.loc[((null_areaType) & (
#     (geocode_df['state'].isin(country_toCheck)))),
#     'areaType'] = 'Country'

In [491]:
# Organizing `areaType` into bins for granularity
areaType_bins = {
    'State': ['State', 'Territory', 'Unorganized Territory', 'Island'],
    'County': ['County', 'County-equivalent', 'Parish', 'Borough', 'Census Area'],
    'City': ['City', 'Township', 'Municipality', 'Municipio', 'Census Subarea', 'CCD'],
    'Native Land': ['Reservation', 'Indian Reservation', 'ANV/ANVSA', 'OTSA', 'TDSA', 'Native Regional Corporation', 'Nett Lake'],
    'Other': ['District', 'Police Jury Ward', 'Sisseton']
}

# Converting bins into reverse-lookup for mapping
# (Treats dictionary created with list-comprehension)
areaType_map = {
    old_type: new_type  # Specifying key:value pair setup
    for new_type, old_types in areaType_bins.items()    # Outer-iteration of type_key -> [type_val1, type_val2, ...]
    for old_type in old_types   # Inner-iteration of type in [type1, type2, type3, ...]
}

# Set mapping to DataFrame for corrections
geocode_df['areaType'] = geocode_df['areaType'].map(areaType_map, na_action = 'ignore')
geocode_df.loc[geocode_df['areaType'].isna()]

,area,areaType,state,id
6,Nez Perce Indian Reservation,NaN,ID,85cccdec-5e22-436f-a3e7-acd7340172cf
11,Colville Indian Reservation,NaN,WA,234f8a8c-91e7-4c3a-8fa4-95a9febd31b7
14,Yakama Reservation,NaN,WA,0a4a121f-2abb-42a9-a200-e0db24d65e01
31,San Carlos Indian Reservation,NaN,AZ,6afdfc89-92bd-4d82-b16d-a1fdb52e341c
40,Yakama Reservation,NaN,WA,7507247f-5c65-41b8-84a8-8dfb2551270f
...,...,...,...,...
59122,Navajo Nation Reservation,NaN,UT,69aebd28-3a22-4ed8-8770-7047d4ffde42
59123,Navajo Nation Reservation,NaN,AZ,430f4854-e0d6-44ec-84cb-9020c699a3a8
59124,Navajo Nation Reservation,NaN,UT,430f4854-e0d6-44ec-84cb-9020c699a3a8
59125,Navajo Nation Reservation,NaN,NM,c44526db-cda6-4f71-96c9-3c6b0eb0d27b


In [ ]:
# `areaType` DATA CORRECTION USING CSV FILE
#-------------------------------------------

# Reading in CSV File for City, County, and State information to correct FEMA area data
place_df = pd.read_csv('Assets/CSVs/us_cities_states_counties.csv', delimiter = '|')
place_df['County'] = place_df['County'].str.title()
place_df = place_df.drop(axis = 0, columns = 'State short')
place_df['City'] = pd.concat([place_df['City'], place_df['City alias']], ignore_index = True)
place_df = place_df.rename(
    columns = {
        'State full': 'State'
    }
)
place_df = place_df[['State', 'County', 'City']]

# Melting place_df for use as a lookup-table in merge with geocode_df
place_key = pd.melt(place_df, var_name = 'areaType', value_name = 'area'
                    ).drop_duplicates('area'
                    ).reset_index(drop = True)
place_key['area'] = place_key['area'].astype(str).str.strip().str.title()

update_df = geocode_df.loc[null_areaType
                        ].drop(axis = 0, columns = 'areaType'
                        ).merge(place_key, how = 'left', on = 'area')

update_df = update_df[['id', 'areaType']].loc[update_df['areaType'].notna()]
geocode_df = geocode_df.merge(update_df, how = 'left', on = 'id', )
# result.loc[result['areaType'].isnull()]

,area,state,region,tribalRequest,id,areaType
0,Colorado,CO,8,0,2244effe-3ea4-418f-a26e-fd490eac37b1,State
2,New Mexico,NM,6,0,033aff9e-cfba-40b2-baa0-dd80fb92687f,State
3,Texas,TX,6,0,921455dc-3309-424f-a39e-978ac0c69eab,State
4,Oklahoma,OK,6,0,de0e25b2-97b3-4c50-b60c-855d3a49887c,State
5,Texas,TX,6,0,fe2a0426-7bdd-485b-9aae-337d0b5fda2f,State
...,...,...,...,...,...,...
1118,Suffolk,VA,3,0,3017cf50-9dce-4470-b42b-36c3ea73438f,County
1119,Virginia Beach,VA,3,0,5dcda410-f418-43db-a3e7-931ec2dfde44,City
1120,Ebeye,MH,9,0,efab5d33-4799-4106-a2db-e25030176791,City
1122,Puerto Rico,PR,2,0,110a368d-b7f8-4a53-8911-7ee27fc58f33,State


In [465]:
geocode_df.iloc[812, 5].split()

['37824e5b-f081-400a-83b5-138efdd9655a']

In [466]:
# Examining what is still Null and tackling each null type as it's revealed
geocode_df.loc[geocode_df['areaType'].isnull(), :].tail(45)

,area,areaType,state,region,tribalRequest,id
57499,Navajo Nation Reservation,NaN,AZ,9,1,c44526db-cda6-4f71-96c9-3c6b0eb0d27b
57548,Indian River,NaN,FL,4,0,a15b935a-51ea-4ab6-bf0a-f9cfabf9e083
57639,Indian River,NaN,FL,4,0,af2901a2-10e7-43ca-8967-d46dba9dc458
57689,Indian River,NaN,FL,4,0,01972c44-814d-4ce4-b114-ed391345733e
57731,Indian River,NaN,FL,4,0,8b5f6ecc-06a5-49b5-8a09-f745e1e05962
57779,Alexandria,NaN,VA,3,0,b63904be-915e-4002-870a-2b52db515698
57780,Chesapeake,NaN,VA,3,0,13a94c69-4452-4cf4-b409-63833409d1be
57781,Colonial Heights,NaN,VA,3,0,530a14bb-8bba-4238-a02e-d477d8d58b4c
57782,Hampton,NaN,VA,3,0,93dc4bb0-1c7d-4350-8c23-952da05bd613
57783,Hopewell,NaN,VA,3,0,22566a27-afbc-467d-8ce3-25f0c34617a1


In [467]:
anv_anvsa_loc = (geocode_df['areaType'] == 'ANV/ANVSA')
geocode_df.loc[anv_anvsa_loc, 'areaType'] = 'Village'
geocode_df.loc[anv_anvsa_loc]

,area,areaType,state,region,tribalRequest,id


In [468]:
# geoapify_base = 'https://api.geoapify.com/v1/geocode/search?'
# geoapify_params = {
#     'apiKey': api_keys.geoapify_key
#     }

# for index, row in fema_disasters.iterrows():
#     geoapify_params['state'] = row['state']
#     geocode_response = requests.get(geoapify_base, geoapify_params).json()
#     pprint(geocode_response)
#     break